## IPL Match Dataset

In [1]:
#pip install wget

Note: you may need to restart the kernel to use updated packages.


In [2]:
#pip install pyarrow

In [4]:
import wget
#wget.download('https://cricsheet.org/downloads/ipl_json.zip')

In [5]:
"""
file_name='ipl_json.zip'
from zipfile import ZipFile
with ZipFile(file_name, 'r') as zip:
    # printing all the contents of the zip file
#     zip.printdir()
  
    # extracting all the files
    print('Extracting all the files now...')
    zip.extractall()
    print('Done!')
"""

Extracting all the files now...
Done!


In [6]:
import json
import pandas as pd
import numpy as np
import bar_chart_race as bcr
import os

### Data Model
### Table with ball-by-ball details
Match_id|Innings|Over_no|Ball_no|Batter|Non-Striker|Bowler|Total_runs|Batter_runs|Extra_runs|Extra_Type|Wickets|Wicket_kind|Player_Out|fielder_caught

In [7]:
cnt = 0
for file in os.listdir('./ipl_json'):
    if file.endswith('.json'):
        #print(file)
        with open('./ipl_json/' + file, 'r') as f:
            json_data = json.load(f)
            match_number_list = []
            innings_list = []
            overs_list = []
            deliveries_list = []
            batters_list = []
            bowlers_list = []
            batters_runs_list = []
            extra_runs_list = []
            dates_list = []
            li=json_data['innings']
            for cnt_innings, val_innings in enumerate(li):
                for cnt_overs, val_overs in enumerate(val_innings['overs']):
                    for cnt_deliveries, val_deliveries in enumerate(val_overs['deliveries']):
                        if 'match_number' in json_data['info']['event'].keys():
                            match_number_list.append(json_data['info']['event']['match_number'])
                        elif 'stage' in json_data['info']['event'].keys():
                            match_number_list.append(json_data['info']['event']['stage'])
                        innings_list.append(str(cnt_innings + 1))
                        overs_list.append(str(cnt_overs))
                        deliveries_list.append(str(cnt_deliveries + 1))
                        batters_list.append(val_deliveries['batter'])
                        bowlers_list.append(val_deliveries['bowler'])
                        batters_runs_list.append(str(val_deliveries['runs']['batter']))
                        extra_runs_list.append(str(val_deliveries['runs']['extras']))
                        dates_list.append(json_data['info']['dates'][0])
            data_dict = {}
            data_dict['dates'] = dates_list
            data_dict['match_number'] = match_number_list
            data_dict['innings'] = innings_list
            data_dict['overs'] = overs_list
            data_dict['deliveries'] = deliveries_list
            data_dict['batters'] = batters_list
            data_dict['batters_runs'] = batters_runs_list
            data_dict['extra_runs'] = extra_runs_list
            data_dict['bowlers'] = bowlers_list
            df = pd.DataFrame.from_dict(data_dict)
            df.to_parquet('ipl',partition_cols = ['match_number','dates'])
#             df.to_csv('file' + str(cnt) + '.csv')
#             cnt = cnt + 1

In [8]:
df = pd.read_parquet('ipl')

In [9]:
df.head()

,innings,overs,deliveries,batters,batters_runs,extra_runs,bowlers,match_number,dates
0,1,0,1,SC Ganguly,0,1,P Kumar,1,2008-04-18
1,1,0,2,BB McCullum,0,0,P Kumar,1,2008-04-18
2,1,0,3,BB McCullum,0,1,P Kumar,1,2008-04-18
3,1,0,4,BB McCullum,0,0,P Kumar,1,2008-04-18
4,1,0,5,BB McCullum,0,0,P Kumar,1,2008-04-18


In [10]:
#!rm *zip

In [11]:
df.shape

(208042, 9)

In [12]:
#df.head(127)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 208042 entries, 0 to 208041
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype   
---  ------        --------------   -----   
 0   innings       208042 non-null  object  
 1   overs         208042 non-null  object  
 2   deliveries    208042 non-null  object  
 3   batters       208042 non-null  object  
 4   batters_runs  208042 non-null  object  
 5   extra_runs    208042 non-null  object  
 6   bowlers       208042 non-null  object  
 7   match_number  208042 non-null  category
 8   dates         208042 non-null  category
dtypes: category(2), object(7)
memory usage: 11.7+ MB


In [14]:
cols = ['innings', 'overs', 'deliveries', 'batters_runs', 'extra_runs']
df[cols] = df[cols].astype('int64')

In [15]:
df['dates'] = pd.to_datetime(df['dates'], infer_datetime_format = True, utc = True).astype('datetime64[ns]')

In [16]:
df.groupby(['dates', 'batters'])['batters_runs'].sum()

dates       batters        
2008-04-18  AA Noffke            9
            B Akhil              0
            BB McCullum        158
            CL White             6
            DJ Hussey           12
                              ... 
2021-10-15  SP Narine            2
            Shakib Al Hasan      0
            Shivam Mavi         20
            Shubman Gill        51
            VR Iyer             50
Name: batters_runs, Length: 13071, dtype: int64

In [17]:
#df[(df.dates == '2008-04-18') & (df.batters == 'BB McCullum')].overs.value_counts()

In [18]:
    df['year'] = df.dates.dt.year
    df['year'] = pd.to_datetime(df['year'],format='%Y')

In [26]:
df.head(20)

,innings,overs,deliveries,batters,batters_runs,extra_runs,bowlers,match_number,dates,year
0,1,0,1,SC Ganguly,0,1,P Kumar,1,2008-04-18,2008-01-01
1,1,0,2,BB McCullum,0,0,P Kumar,1,2008-04-18,2008-01-01
2,1,0,3,BB McCullum,0,1,P Kumar,1,2008-04-18,2008-01-01
3,1,0,4,BB McCullum,0,0,P Kumar,1,2008-04-18,2008-01-01
4,1,0,5,BB McCullum,0,0,P Kumar,1,2008-04-18,2008-01-01
5,1,0,6,BB McCullum,0,0,P Kumar,1,2008-04-18,2008-01-01
6,1,0,7,BB McCullum,0,1,P Kumar,1,2008-04-18,2008-01-01
7,1,1,1,BB McCullum,0,0,Z Khan,1,2008-04-18,2008-01-01
8,1,1,2,BB McCullum,4,0,Z Khan,1,2008-04-18,2008-01-01
9,1,1,3,BB McCullum,4,0,Z Khan,1,2008-04-18,2008-01-01


In [20]:
df.groupby(['batters'])[['batters_runs']].sum().sort_values(by='batters_runs', ascending=False)[:10]

,batters_runs
batters,
V Kohli,6293
S Dhawan,5784
RG Sharma,5613
SK Raina,5536
DA Warner,5451
AB de Villiers,5181
CH Gayle,4997
MS Dhoni,4746
RV Uthappa,4724


In [21]:
df.groupby(['year','batters'])[['batters_runs']].sum()

batters_runs
year       batters                        
2008-01-01 A Chopra                     42
           A Kumble                     13
           A Mishra                     37
           A Mukund                      0
           A Nehra                       3
...                                    ...
2021-01-01 Virat Singh                  15
           WP Saha                     131
           Washington Sundar            31
           YBK Jaiswal                 249
           YS Chahal                    10

[2095 rows x 1 columns]

In [22]:
df_ipl = df.groupby(['year', 'batters']).size().groupby(level=1).cumsum().unstack().fillna(method='ffill')
df_ipl.fillna(0)

batters,A Ashish Reddy,A Chandila,A Chopra,A Choudhary,A Dananjaya,A Flintoff,A Kumble,A Mishra,A Mithun,A Mukund,...,Y Venugopal Rao,YA Abdulla,YBK Jaiswal,YK Pathan,YS Chahal,YV Takawale,Yashpal Singh,Younis Khan,Yuvraj Singh,Z Khan
year,,,,,,,,,,,,,,,,,,,,,
2008-01-01,0.0,0.0,55.0,0.0,0.0,0.0,17.0,42.0,0.0,1.0,...,217.0,0.0,0.0,253.0,0.0,69.0,0.0,7.0,191.0,43.0
2009-01-01,0.0,0.0,75.0,0.0,0.0,57.0,37.0,62.0,0.0,1.0,...,335.0,0.0,0.0,445.0,0.0,91.0,67.0,7.0,508.0,54.0
2010-01-01,0.0,0.0,75.0,0.0,0.0,57.0,49.0,116.0,4.0,1.0,...,379.0,1.0,0.0,655.0,0.0,91.0,67.0,7.0,714.0,67.0
2011-01-01,0.0,0.0,75.0,0.0,0.0,57.0,49.0,168.0,24.0,1.0,...,671.0,1.0,0.0,860.0,0.0,91.0,67.0,7.0,983.0,91.0
2012-01-01,30.0,1.0,75.0,0.0,0.0,57.0,49.0,189.0,24.0,1.0,...,800.0,1.0,0.0,1039.0,0.0,91.0,67.0,7.0,983.0,108.0
2013-01-01,120.0,7.0,75.0,0.0,0.0,57.0,49.0,292.0,26.0,23.0,...,800.0,1.0,0.0,1286.0,0.0,91.0,67.0,7.0,1176.0,108.0
2014-01-01,120.0,7.0,75.0,0.0,0.0,57.0,49.0,299.0,26.0,23.0,...,865.0,1.0,0.0,1454.0,2.0,183.0,67.0,7.0,1464.0,115.0
2015-01-01,166.0,7.0,75.0,0.0,0.0,57.0,49.0,312.0,26.0,23.0,...,865.0,1.0,0.0,1697.0,3.0,183.0,67.0,7.0,1679.0,115.0
2016-01-01,196.0,7.0,75.0,0.0,0.0,57.0,49.0,326.0,26.0,23.0,...,865.0,1.0,0.0,1953.0,3.0,183.0,67.0,7.0,1862.0,128.0


In [23]:
df_top25_batters = df.groupby(['batters'])[['batters_runs']].sum().sort_values(by='batters_runs', ascending=False)[:25]
df_top25_batters.index

Index(['V Kohli', 'S Dhawan', 'RG Sharma', 'SK Raina', 'DA Warner',
       'AB de Villiers', 'CH Gayle', 'MS Dhoni', 'RV Uthappa', 'G Gambhir',
       'KD Karthik', 'AM Rahane', 'AT Rayudu', 'SR Watson', 'MK Pandey',
       'KA Pollard', 'KL Rahul', 'YK Pathan', 'SV Samson', 'F du Plessis',
       'BB McCullum', 'PA Patel', 'Yuvraj Singh', 'V Sehwag', 'M Vijay'],
      dtype='object', name='batters')

In [24]:
df_top25_runs = df_ipl[df_top25_batters.index]

In [25]:
df_top25_runs

batters,V Kohli,S Dhawan,RG Sharma,SK Raina,DA Warner,AB de Villiers,CH Gayle,MS Dhoni,RV Uthappa,G Gambhir,...,KA Pollard,KL Rahul,YK Pathan,SV Samson,F du Plessis,BB McCullum,PA Patel,Yuvraj Singh,V Sehwag,M Vijay
year,,,,,,,,,,,,,,,,,,,,,
2008-01-01,168.0,308.0,276.0,304.0,NaN,100.0,NaN,319.0,291.0,394.0,...,NaN,NaN,253.0,NaN,NaN,96.0,315.0,191.0,229.0,NaN
2009-01-01,393.0,354.0,599.0,622.0,137.0,459.0,159.0,589.0,468.0,677.0,...,NaN,NaN,445.0,NaN,NaN,344.0,446.0,508.0,375.0,69.0
2010-01-01,609.0,534.0,909.0,1009.0,332.0,582.0,356.0,804.0,692.0,900.0,...,152.0,NaN,655.0,NaN,NaN,461.0,529.0,714.0,602.0,374.0
2011-01-01,1082.0,849.0,1213.0,1344.0,613.0,830.0,698.0,1051.0,907.0,1232.0,...,293.0,NaN,860.0,NaN,NaN,753.0,718.0,983.0,849.0,723.0
2012-01-01,1415.0,1297.0,1563.0,1679.0,774.0,1033.0,1170.0,1334.0,1264.0,1661.0,...,456.0,NaN,1039.0,NaN,310.0,1040.0,884.0,983.0,1166.0,996.0
2013-01-01,1886.0,1559.0,1978.0,2049.0,1107.0,1259.0,1654.0,1632.0,1649.0,2015.0,...,746.0,16.0,1286.0,179.0,310.0,1047.0,1145.0,1176.0,1410.0,1289.0
2014-01-01,2187.0,1891.0,2285.0,2419.0,1493.0,1514.0,1842.0,1890.0,2134.0,2316.0,...,954.0,182.0,1454.0,459.0,550.0,1388.0,1336.0,1464.0,1744.0,1486.0
2015-01-01,2582.0,2184.0,2624.0,2738.0,1861.0,1816.0,2201.0,2199.0,2421.0,2604.0,...,1224.0,315.0,1697.0,625.0,867.0,1681.0,1591.0,1679.0,1833.0,1719.0
2016-01-01,3237.0,2622.0,2996.0,3059.0,2440.0,2231.0,2360.0,2419.0,2721.0,3028.0,...,1380.0,592.0,1953.0,890.0,1036.0,1954.0,1752.0,1862.0,1833.0,2092.0


In [37]:
bcr.bar_chart_race(df=df_top25_runs, title = 'IPL Top Scorers', period_length = 3000, steps_per_period=50, period_fmt='%Y', 
                   dpi=200, interpolate_period=True, filename = 'ipl_top_scorers.mp4')